In [5]:
!pip install qrcode


^C


In [ ]:
import os
import qrcode
from PIL import Image, ImageDraw, ImageFont
from datetime import date
import base64
import io
import random

# ======= CONFIGURACIÓN PERSONALIZABLE =======
# Configuración de colores
PRIMARY_COLOR = (25, 52, 95) 
SECONDARY_COLOR = (212, 175, 55) 
TEXT_COLOR = (40, 40, 45)         
HIGHLIGHT_COLOR = (145, 30, 30)  

# Configuración de degradado de fondo
GRADIENT_TOP_COLOR = (240, 248, 255)
GRADIENT_BOTTOM_COLOR = (220, 230, 242)

# Configuración de diseño
CERTIFICATE_WIDTH = 1300
CERTIFICATE_HEIGHT = 950
BORDER_WIDTH = 5
CORNER_SIZE = 40
CORNER_LINE_WIDTH = 3
TEXTURE_OPACITY = 10

# Configuración de logos
LOGO_HEIGHT = 85
LOGO_Y_POSITION = 45

# Configuración de QR
QR_SIZE = 130
QR_FILL_COLOR = "navy"
QR_BACK_COLOR = "white"
QR_PREFIX = "USC"  # Prefijo para códigos de certificado

# Configuración de textos
TITLE_TEXT = "UNIVERSIDAD SIDERAL CARRIÓN"
CERTIFICA_TEXT = "Certifica que"
DESCRIPTION_LINES = [
    "Ha culminado satisfactoriamente el",
    "BOOTCAMP DE CIENCIA DE DATOS",
    "con una duración de 600 horas intensivas, desarrollando competencias en",
    "análisis de datos, inteligencia artificial, inteligencia artesanal, y programacion avanzada"
]
FOOTER_TEXT = "Universidad Sideral Carrión - Excelencia Académica Internacional"

# Configuración de firmas
SIGNATURE_WIDTH = 280
SIGNATURE_DATA = [
    {
        "name": "PHd. SIDERAL LUJAN CARRION",
        "title": "Rector"
    },
    {
        "name": "Mgtr. MAKANAKI LA REALEZA LUJAN CARRION", 
        "title": "Vicerrector Académico"
    }
]

# Rutas de archivos de imagen

ASSETS_PATH = "/kaggle/input/data-usc/assets/"
BACKGROUND_IMAGE = ASSETS_PATH + "fondo.png"
LOGO_PATHS = [ASSETS_PATH + f"logo{i}.png" for i in range(3)]

# Configuración de salida
OUTPUT_QUALITY = 95
OUTPUT_DPI = (300, 300)

# ======= FUNCIONES =======
def generate_qr_code(data):
    """Genera un código QR con el texto dado y lo devuelve como imagen PIL."""
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_H,
        box_size=10,
        border=4,
    )
    qr.add_data(data)
    qr.make(fit=True)
    img = qr.make_image(fill_color=QR_FILL_COLOR, back_color=QR_BACK_COLOR)
    return img

def load_image(image_path):
    try:
        return Image.open(image_path)
    except Exception:
        # Crear una imagen vacía si no se encuentra el archivo
        return Image.new('RGBA', (100, 100), color=(255, 255, 255, 0))

def add_gradient_background(image, top_color=GRADIENT_TOP_COLOR, bottom_color=GRADIENT_BOTTOM_COLOR):
    width, height = image.size
    for y in range(height):
        # Calcula el color para esta línea basado en la posición vertical
        r = int(top_color[0] * (1 - y/height) + bottom_color[0] * (y/height))
        g = int(top_color[1] * (1 - y/height) + bottom_color[1] * (y/height))
        b = int(top_color[2] * (1 - y/height) + bottom_color[2] * (y/height))

        # Dibuja una línea horizontal con este color
        for x in range(width):
            if image.getpixel((x, y))[0:3] == (255, 255, 255):  # Solo si es blanco
                image.putpixel((x, y), (r, g, b))

    return image

def add_texture(image, opacity=TEXTURE_OPACITY):
    width, height = image.size
    for x in range(0, width, 4):
        for y in range(0, height, 4):
            if random.randint(0, 100) < opacity:
                # Sólo aplicar a píxeles que no sean completamente blancos (para no afectar logos)
                pixel = image.getpixel((x, y))
                if isinstance(pixel, tuple) and len(pixel) >= 3 and pixel[0:3] != (0, 0, 0):
                    r, g, b = pixel[0:3]
                    variation = random.randint(-5, 5)
                    r = max(0, min(255, r + variation))
                    g = max(0, min(255, g + variation))
                    b = max(0, min(255, b + variation))

                    if len(pixel) == 4:  # Si tiene canal alfa
                        image.putpixel((x, y), (r, g, b, pixel[3]))
                    else:
                        image.putpixel((x, y), (r, g, b))

    return image

def add_decorative_corners(draw, width, height, color=SECONDARY_COLOR, size=CORNER_SIZE, line_width=CORNER_LINE_WIDTH):
    """Añade esquinas decorativas al certificado."""
    # Esquina superior izquierda
    draw.line([(40, 40), (40, 40 + size)], fill=color, width=line_width)
    draw.line([(40, 40), (40 + size, 40)], fill=color, width=line_width)

    # Esquina superior derecha
    draw.line([(width - 40, 40), (width - 40, 40 + size)], fill=color, width=line_width)
    draw.line([(width - 40, 40), (width - 40 - size, 40)], fill=color, width=line_width)

    # Esquina inferior izquierda
    draw.line([(40, height - 40), (40, height - 40 - size)], fill=color, width=line_width)
    draw.line([(40, height - 40), (40 + size, height - 40)], fill=color, width=line_width)

    # Esquina inferior derecha
    draw.line([(width - 40, height - 40), (width - 40, height - 40 - size)], fill=color, width=line_width)
    draw.line([(width - 40, height - 40), (width - 40 - size, height - 40)], fill=color, width=line_width)

def generate_unique_code(student_name, prefix=QR_PREFIX):
    """Genera un código único para el certificado basado en el nombre y la fecha."""
    today = date.today().strftime("%Y%m%d")
    name_hash = hash(student_name) % 10000
    return f"{prefix}-{today}-{name_hash:04d}"

def add_watermark(certificate, bg_image_path=BACKGROUND_IMAGE):
    """Añade una marca de agua transparente al fondo del certificado"""
    width, height = certificate.size
    
    # Cargar imagen de fondo
    escudo = load_image(bg_image_path)
    
    # Redimensionar marca de agua
    escudo_width = width - 250
    escudo_height = int(escudo.height * escudo_width / escudo.width)
    escudo = escudo.resize((escudo_width, escudo_height), Image.LANCZOS)

    # Crear una versión transparente del escudo
    escudo = escudo.convert("RGBA")
    data = escudo.getdata()
    new_data = []
    for item in data:
        new_data.append((item[0], item[1], item[2], 20)) 
    escudo.putdata(new_data)

    # Pegar escudo como marca de agua
    escudo_position = ((width - escudo_width) // 2, (height - escudo_height) // 2)
    certificate.paste(escudo, escudo_position, escudo)
    
    return certificate

def add_borders(draw, width, height, primary_color=PRIMARY_COLOR, secondary_color=SECONDARY_COLOR, border_width=BORDER_WIDTH):
    """Añade bordes decorativos al certificado"""
    # Borde exterior
    for i in range(border_width):
        draw.rectangle([(20+i, 20+i), (width-20-i, height-20-i)], outline=primary_color)

    # Borde interior dorado
    for i in range(2):  # Más delgado
        draw.rectangle([(30+i, 30+i), (width-30-i, height-30-i)], outline=secondary_color)
    
    return draw

def add_logos(certificate, logo_paths=LOGO_PATHS, logo_height=LOGO_HEIGHT, logo_y=LOGO_Y_POSITION):
    """Añade los logos al certificado"""
    width = certificate.width
    
    # Cargar logos
    logos = [load_image(path) for path in logo_paths]
    
    # Redimensionar logos para la cabecera
    resized_logos = []
    for logo in logos:
        resized = logo.resize((int(logo.width * logo_height / logo.height), logo_height), Image.LANCZOS)
        resized_logos.append(resized)
    
    # Posiciones para los logos
    logo0_x = 50
    logo2_x = width - 50 - resized_logos[2].width
    logo1_x = (width - resized_logos[1].width) // 2

    # Pegar logos
    logo_positions = [logo0_x, logo1_x, logo2_x]
    for i, logo in enumerate(resized_logos):
        certificate.paste(logo, (logo_positions[i], logo_y), logo if logo.mode == 'RGBA' else None)
    
    return certificate

def add_qr_code(certificate, code, size=QR_SIZE, color=SECONDARY_COLOR):
    """Añade un código QR al certificado"""
    width, height = certificate.size
    
    # Generar código QR
    qr_img = generate_qr_code(code)
    qr_img = qr_img.resize((size, size))

    # Añadir un pequeño marco alrededor del QR
    qr_border = Image.new('RGB', (size + 12, size + 12), color=color)
    qr_border.paste(qr_img, (6, 6))

    # Colocar en la esquina inferior derecha
    certificate.paste(qr_border, (width - size - 55, height - size - 55))
    
    return certificate

def format_date():
    month_names = {
        1: "enero", 2: "febrero", 3: "marzo", 4: "abril", 5: "mayo", 6: "junio",
        7: "julio", 8: "agosto", 9: "septiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
    }
    today = date.today()
    return f"Lima, {today.day} de {month_names[today.month]} del {today.year}"

def generate_certificate(student_name, config=None):
    # Si se proporciona una configuración personalizada, actualiza las variables
    if config:
        pass
    
    # Configuración del tamaño del certificado
    width, height = CERTIFICATE_WIDTH, CERTIFICATE_HEIGHT
    certificate = Image.new('RGB', (width, height), color='white')

    # Aplicar fondo con degradado
    certificate = add_gradient_background(certificate)

    # Añadir textura sutil
    certificate = add_texture(certificate)

    # Añadir marca de agua
    certificate = add_watermark(certificate)

    draw = ImageDraw.Draw(certificate)

    # Añadir bordes
    draw = add_borders(draw, width, height)

    # Añadir esquinas decorativas
    add_decorative_corners(draw, width, height)

    # Añadir logos
    certificate = add_logos(certificate)

    # Cargar fuentes
    title_font = ImageFont.load_default()
    name_font = ImageFont.load_default()
    normal_font = ImageFont.load_default()
    bootcamp_font = ImageFont.load_default()
    small_font = ImageFont.load_default()
    signature_font = ImageFont.load_default()
    footer_font = ImageFont.load_default()

    # Título y contenido principal
    title_width = draw.textlength(TITLE_TEXT, font=title_font)
    draw.text(((width - title_width) // 2, 180), TITLE_TEXT, font=title_font, fill=PRIMARY_COLOR)

    # Agregar línea decorativa bajo el título
    line_y = 245
    draw.line([(width//2 - 220, line_y), (width//2 + 220, line_y)], fill=SECONDARY_COLOR, width=3)

    # Certificado
    certifica_width = draw.textlength(CERTIFICA_TEXT, font=normal_font)
    draw.text(((width - certifica_width) // 2, 275), CERTIFICA_TEXT, font=normal_font, fill=TEXT_COLOR)

    # Nombre del estudiante
    name_width = draw.textlength(student_name.upper(), font=name_font)
    name_y = 320
    draw.text(((width - name_width) // 2, name_y), student_name.upper(), font=name_font, fill=HIGHLIGHT_COLOR)

    # Descripción del curso con estilo mejorado
    line_y = 400
    for i, line in enumerate(DESCRIPTION_LINES):
        font = normal_font
        color = TEXT_COLOR

        if i == 1:  # Para el nombre del bootcamp (destacado)
            font = bootcamp_font
            color = PRIMARY_COLOR

            # Añadir un subrayado sutil al nombre del bootcamp
            text_width = draw.textlength(line, font=font)
            text_x = (width - text_width) // 2
            text_bottom = line_y + 45
            draw.line([(text_x, text_bottom), (text_x + text_width, text_bottom)],
                      fill=SECONDARY_COLOR, width=1)

        line_width = draw.textlength(line, font=font)
        draw.text(((width - line_width) // 2, line_y), line, font=font, fill=color)
        line_y += 40 if i != 1 else 60

    # Fecha con formato mejorado
    today_text = format_date()
    date_width = draw.textlength(today_text, font=small_font)
    draw.text(((width - date_width) // 2, line_y + 20), today_text, font=small_font, fill=TEXT_COLOR)

    # Firmas con estilo mejorado
    signature_y = 630

    # Añadir firmas
    for i, sig_data in enumerate(SIGNATURE_DATA):
        sig_x = (i + 1) * width // (len(SIGNATURE_DATA) + 1) - SIGNATURE_WIDTH // 2
        draw.line([(sig_x, signature_y), (sig_x + SIGNATURE_WIDTH, signature_y)], fill=PRIMARY_COLOR, width=1)

        sig_name = sig_data["name"]
        sig_title = sig_data["title"]
        sig_name_width = draw.textlength(sig_name, font=signature_font)
        sig_title_width = draw.textlength(sig_title, font=signature_font)

        draw.text((sig_x + (SIGNATURE_WIDTH - sig_name_width) // 2, signature_y + 15),
                sig_name, font=signature_font, fill=PRIMARY_COLOR)
        draw.text((sig_x + (SIGNATURE_WIDTH - sig_title_width) // 2, signature_y + 40),
                sig_title, font=signature_font, fill=TEXT_COLOR)

    # Generar código único para el certificado
    certificate_code = generate_unique_code(student_name)

    # Pie de página con estilo mejorado
    footer_width = draw.textlength(FOOTER_TEXT, font=footer_font)
    draw.text(((width - footer_width) // 2, height - 70), FOOTER_TEXT, font=footer_font, fill=PRIMARY_COLOR)

    # Añadir código del certificado
    code_text = f"Código de verificación: {certificate_code}"
    code_width = draw.textlength(code_text, font=small_font)
    draw.text(((width - code_width) // 2, height - 40), code_text, font=small_font, fill=TEXT_COLOR)

    # Añadir código QR
    certificate = add_qr_code(certificate, certificate_code)

    # Nombre del archivo de salida
    output_filename = f"certificado_{student_name.lower().replace(' ', '_')}.png"

    # Guardar imagen con mayor calidad
    certificate.save(output_filename, quality=OUTPUT_QUALITY, dpi=OUTPUT_DPI)

    return output_filename

def main():
    # Solicitar nombre del estudiante
    student_name = input("Nombre del estudiante: ")

    if not student_name:
        print("Error: Nombre vacío.")
        return

    try:
        certificate_file = generate_certificate(student_name)
        print(f"Certificado generado: {certificate_file}")
        print("¡Listo! El certificado ha sido creado")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()